Import Package dan Dataset

In [ ]:
import re
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
import string
import nltk
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
%matplotlib inline

In [ ]:
test=pd.read_excel('drive/My Drive/Data BDC - Satria Data 2020/Data Uji/Data Uji BDC.xlsx')
train=pd.read_excel('drive/My Drive/Data BDC - Satria Data 2020/Data Latih/Data Latih BDC.xlsx')
testoriginal=test.copy()
trainoriginal=train.copy()
dataset=train.copy()
dataset=dataset.append(test)
dataset=dataset.reset_index()
dataset

Penghilangan simbol dan perubahan menjadi huruf kecil pada kolom narasi dan judul

In [ ]:
#ilangin simbol
dataset['narasibersih'] = dataset['narasi'].str.replace("[^a-zA-Z#]", " ")
dataset['narasibersih'] = dataset['narasibersih'].str.lower()
dataset['judulbersih'] = dataset['judul'].str.replace("[^a-zA-Z#]", " ")
dataset['judulbersih'] = dataset['judulbersih'].str.lower()
dataset.head(10)

Import Package untuk Penerjemahan data

In [ ]:
pip install googletrans

In [ ]:
from googletrans import Translator
import time
translator = Translator()
narasi=dataset['narasibersih']
judul=dataset['judulbersih']

In [ ]:
for i in range (len(narasi)):
  translation = translator.translate(narasi[i], dest='id')
  if translation.src=='en':
    dataset.iloc[[i],[5]]=translation.text
    print(i, translation.origin, '->',translation.text)
  if i%400==0:
    time.sleep(10)
    translator = Translator()

In [ ]:
for i in range (len(judul)):
  translation = translator.translate(judul[i], dest='id')
  if translation.src=='en':
    dataset.iloc[[i],[6]]=translation.text
    print(i, translation.origin, '->',translation.text)
  if i%400==0:
    time.sleep(10)
    translator = Translator()

Menghapus kata yang terdiri dari 2 huruf atau kurang serta tokenisasi

In [ ]:
#lowercase
dataset['narasibersih'] = dataset['narasibersih'].str.lower()
dataset['judulbersih'] = dataset['judulbersih'].str.lower()
#ilangin kata yang panjangnya kurang dari 2 huruf
dataset['narasibersih'] = dataset['narasibersih'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
dataset['judulbersih'] = dataset['judulbersih'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>2]))
dataset.head()

In [ ]:
#tokenize 
tokenized_narasi = dataset['narasibersih'].apply(lambda x: x.split())
tokenized_judul = dataset['judulbersih'].apply(lambda x: x.split())
tokenized_judul.head()

In [ ]:
tokenized_narasi.head()

In [ ]:
print(len(tokenized_narasi))
print(len(tokenized_judul))

Autocorrect

In [ ]:
import io
import time
from datetime import timedelta
import logging
logging.basicConfig(level=logging.INFO)
import gensim
from gensim.models.wrappers import FastText
import shutil

In [ ]:
#downloading model
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz

In [ ]:
import os
os.mkdir('model')

In [ ]:
shutil.copy2('drive/My Drive/cc.id.300.bin.gz','model')

In [ ]:
!gunzip 'model/cc.id.300.bin.gz'

In [ ]:
model=FastText.load_fasttext_format('model/cc.id.300.bin')

In [ ]:
words=list(model.wv.vocab)
w_rank={}
for i,word in enumerate (words):
  w_rank[word]=i
WORDS=w_rank

In [ ]:
import itertools
dict(itertools.islice(WORDS.items(),10))

In [ ]:
import re
from collections import Counter

def words(text): return re.findall(r'\w+',text.lower())

def P(word):
  return - WORDS.get(word,0)

def correction(word):
  return max(candidates(word),key=P)

def candidates(word):
  return (known([word]) or known(edits1(word)) or known (edits2(word)) or [word])

def known(words):
  return set(w for w in words if w in WORDS)

def edits1(word):
  letters='abcdefghijklmnopqrstuvwxyz'
  splits=[(word[:i], word[i:]) for i in range(len(word)+1)]
  deletes=[L+R[1:] for L, R in splits if R]
  transposes=[L+R[1]+R[0]+R[2:] for L, R in splits if len(R)>1]
  replaces=[L+c+R[1:] for L, R in splits if R for c in letters]
  inserts=[L+c+R for L, R in splits for c in letters]
  return set(deletes+transposes+replaces+inserts)

def edits2(word):
  return(e2 for e1 in edits1(word) for e2 in edits1(e1))

In [ ]:
#Autocorrection
for i in range(len(tokenized_narasi)):
  katadasar=[]
  kalimat = tokenized_narasi[i]
  for j in kalimat:
    katadasar.append(correction(j))
  print('baris ke-',i)
  tokenized_narasi[i] = katadasar
tokenized_narasi.head()

In [ ]:
#Autocorrection
for i in range(len(tokenized_judul)):
  katadasar=[]
  kalimat = tokenized_judul[i]
  for j in kalimat:
    katadasar.append(correction(j))
  print('baris ke-',i)
  tokenized_judul[i] = katadasar
tokenized_judul.head()

In [ ]:
tokenized_narasi.head()

In [ ]:
tokenized_judul.head()

In [ ]:
for i in range(len(tokenized_narasi)):
    tokenized_narasi[i] = ' '.join(tokenized_narasi[i])
for i in range(len(tokenized_judul)):
    tokenized_judul[i] = ' '.join(tokenized_judul[i])

dataset['narasibersih'] = tokenized_narasi
dataset['judulbersih'] = tokenized_judul
dataset.head()

Fitur Ekstraksi Bag of Words

In [ ]:
# Download Stopwords indonesia
nltk.download('stopwords')
swords=nltk.corpus.stopwords.words('indonesian')

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=3, max_features=1000, stop_words = swords)
bow1 = bow_vectorizer.fit_transform(dataset['narasibersih'])
print(bow_vectorizer.get_feature_names())
bow2 = bow_vectorizer.fit_transform(dataset['judulbersih'])
print(bow_vectorizer.get_feature_names())

In [ ]:
df_bulan=pd.DataFrame(arrbulantest)
df_tanggal=pd.DataFrame(arrtanggaltest)
df_tahun=pd.DataFrame(arrtahuntest)
df_bulan.shape

In [ ]:
df_bow1 = pd.DataFrame(bow1.todense())
df_bow2 = pd.DataFrame(bow2.todense())
df_bow=pd.concat([df_tanggal,df_bulan,df_tahun,df_bow1,df_bow2],axis=1)
df_bow

In [ ]:
from sklearn.preprocessing import StandardScaler
sc_X = StandardScaler()
df_bow=sc_X.fit_transform(df_bow)

In [ ]:
dfbowtest=df_bow[4231:,:]
dfbowtrain=df_bow[:4231,:]
print(len(dfbowtest))
print(len(dfbowtrain))
trainlabel=pd.DataFrame(train['label'])